In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import keras
import sklearn

plt.style.use('seaborn') # matplotlib의 기본 scheme 말고 seaborn scheme을 세팅
sns.set(font_scale=2.5) # 폰트 크기 지정 없어 seaborn의 fort_scale을 사용

import missingno as msno

import warnings
warnings.filterwarnings('ignore') # 워닝 메세지 생략

%matplotlib inline

In [3]:
os.listdir("./input")
# train.csv : 학습에 이용하는 파일
# test.csv : 우리가 학습한 모델을 가지고 예측해야하는 파일
# sample_submission.csv : 예측하고 해당 파일 폼에 맞추어 캐글에 체출해야 함(없음)

['gender_submission.csv', 'test.csv', 'train.csv']

In [9]:
# 판다스로 각 파일 로드

df_train = pd.read_csv("./input/train.csv")
df_test = pd.read_csv("./input/test.csv")
#df_submit = pd.read_csv("./input/sample_submission.csv")
df_submit = pd.read_csv("./input/gender_submission.csv") # 남녀의 사망비율

In [10]:
df_train.shape, df_test.shape, df_submit.shape # .shape 속성으로 데이터의 행, 열 크기 확인

((891, 12), (418, 11), (418, 2))

In [11]:
df_train.columns # .columns 속성으로 해당 데이터프레임을 구성하는 컬럼명 확인
# 컬럼은 총 12개로 예측할 feature는 Survived

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [12]:
df_submit.columns
# submit 파일은 2개의 칼럼만 있으면 됌
# 어떤 사람인지 구분하는 PassengerID와 해당 사람이 죽었는지 살앗는지 구별하는 Survived

Index(['PassengerId', 'Survived'], dtype='object')

In [13]:
df_train.head() # .head)() 함수는 불러온 데이터의 상위 5개 행을 출력
# 비슷하게 .tail()는 하위 5개 행을 출력

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [14]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [15]:
df_submit.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [16]:
# 우리 문제의 feature(특색)은 Pclass, Age, SibSp, Parch, Fare이며
# 예측하려는 target label은 Survived임.

#타이타닉 데이터 Feature 설명

# survival : 생존 유무, target 값(0 = 사망, 1 = 생존)
# pclass : 티켓 클래스 (1 = 1sp, 2 = 2nd, 3 = 3rd)
# sex : 성별
# Age : 나이
# sibsp : 함께 탑승한 형제자매, 배우자 총합
# parch : 함계 탑승한 부모, 자녀 수 총합
# ticket : 티켓 넘버
# fare : 탑승 요금
# cabin : 객실 번호
# bembarked : 탑승 항구

In [17]:
df_train.dtypes # .dtype : 열을 기준으로 데이터 형태 반환

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [18]:
df_train.describe()  # .describe() : 각 feature(레이블)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200
